In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
import json

def my_read_json_file(file_name, limit):
    """Read the JSON file and print only the first limit rows"""

    # Open the JSON file
    with open(file_name, "r") as json_file:
        # Load the data as a list of dictionaries
        json_data = json.load(json_file)

    # Print the first `limit` rows
    for i, row in enumerate(json_data[:limit]):
        print(row)

    print("\nPrinted", min(limit, len(json_data)), "lines of", len(json_data), "total lines.")


In [14]:
connection.rollback()

query = """
CREATE TABLE stores (
  store_id INT PRIMARY KEY,
  street VARCHAR(100),
  city VARCHAR(50),
  state CHAR(2),
  zip CHAR(5),
  latitude NUMERIC(9, 6),
  longitude NUMERIC(9, 6)
);

-- Table for customer data
CREATE TABLE customers (
  customer_id INT PRIMARY KEY,
  first_name VARCHAR(50),
  last_name VARCHAR(50),
  first_last_name VARCHAR(100),
  last_first_name VARCHAR(100),
  street VARCHAR(100),
  city VARCHAR(50),
  state CHAR(2),
  zip CHAR(5),
  population INT,
  area NUMERIC(6, 4),
  density NUMERIC(10, 2),
  closest_store_id INT,
  distance NUMERIC(5, 2)
);

-- Table for sales data
CREATE TABLE sales (
  receipt_number VARCHAR(20) PRIMARY KEY,
  customer_id INT,
  sale_date DATE,
  sub_total NUMERIC(8, 2),
  tax NUMERIC(8, 2),
  total_amount NUMERIC(8, 2),
  store_id INT,
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (store_id) REFERENCES stores(store_id)
);

-- Table for line items in each sale
CREATE TABLE line_items (
  line_item_id SERIAL PRIMARY KEY,
  receipt_number VARCHAR(20),
  product_id INT,
  description VARCHAR(100),
  price NUMERIC(8, 2),
  quantity INT,
  line_total NUMERIC(8, 2),
  FOREIGN KEY (receipt_number) REFERENCES sales(receipt_number)
);
"""

cursor.execute(query)


connection.commit()


In [7]:
from neo4j import GraphDatabase
import json
import neo4j

# Neo4j connection details
uri = "bolt://localhost:7687"
user = "neo4j"
password = "ucb_mids_w205"

# Load JSON data
with open('proj3temp_customers_nested.json') as f:
    data = json.load(f)

# Neo4j driver
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

def load_data(tx, customer):
    # Merge Customer
    tx.run("""
        MERGE (c:Customer {customer_id: $customer_id})
        SET c += $customer_props
    """, customer_id=customer['customer_id'], customer_props={
        key: customer[key] for key in customer if key != 'customer_id' and key != 'sales'
    })

    for sale in customer['sales']:
        # Merge Store
        store = sale['store']
        tx.run("""
            MERGE (s:Store {store_id: $store_id})
            SET s += $store_props
        """, store_id=store['store_id'], store_props=store)

        # Merge Sale
        tx.run("""
            MERGE (sa:Sale {receipt_number: $receipt_number})
            SET sa += $sale_props
        """, receipt_number=sale['receipt_number'], sale_props={
            key: sale[key] for key in sale if key not in ['receipt_number', 'store', 'line_items']
        })

        # Create Relationships
        tx.run("""
            MATCH (c:Customer {customer_id: $customer_id})
            MATCH (sa:Sale {receipt_number: $receipt_number})
            MERGE (c)-[:PURCHASED]->(sa)
        """, customer_id=customer['customer_id'], receipt_number=sale['receipt_number'])

        tx.run("""
            MATCH (sa:Sale {receipt_number: $receipt_number})
            MATCH (s:Store {store_id: $store_id})
            MERGE (sa)-[:AT_STORE]->(s)
        """, receipt_number=sale['receipt_number'], store_id=store['store_id'])

        for item in sale['line_items']:
            # Merge Product
            tx.run("""
                MERGE (p:Product {product_id: $product_id})
                SET p.description = $description
            """, product_id=item['product_id'], description=item['description'])

            # Create LineItem relationship
            tx.run("""
                MATCH (sa:Sale {receipt_number: $receipt_number})
                MATCH (p:Product {product_id: $product_id})
                MERGE (sa)-[li:CONTAINS]->(p)
                SET li.price = $price,
                    li.quantity = $quantity,
                    li.line_total = $line_total
            """, receipt_number=sale['receipt_number'], product_id=item['product_id'],
            price=item['price'], quantity=item['quantity'], line_total=item['line_total'])

with driver.session() as session:
    for customer in data['customers']:
        session.write_transaction(load_data, customer)

driver.close()


/tmp/ipykernel_43/4265172927.py:76: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(load_data, customer)
